# GraphGuard

***Locate and find Classes in Apks with updated Obfuscation Mapping***


Processing Steps:
1. String Strategy (Classes + Methods) \
  Strategy using 2 different variants only involving Strings:
  * String Counter: used in Classes and Methods and try to find exact matching counter.
  * Unique Strings: Match Classes by identifying Strings used only in this single Class.



2. Structure Strategy (Classes) \
  Strategy that enumerates all classes and tries to find an optimal match by using the following (weighted) criteria:
  * Modifiers of class
  * Modifiers, Parameters, Parameter Types and Return Types of Methods
  * Number and Types of Fields.



3. Method Strategy (Methods) \
  Strategy that uses already matched classes and tries to find an optimal Method Match in the class by using the following (weighted) criteria:
  * Modifiers
  * Return Type, Parameter Types
  * Bytecode Length
  * References to and from


4. Field Strategy (Fields) \
  Strategy that uses already matched classes and tries to find an optimal Field Match in the class by using the following (weighted criteria)
  * By Type if a matching Type has been found
  * By weighted sub-criteria such as:
    * Access Flags
    * Size
    * Number of reads & Number of writes (references)

***If you use Akrolyb, please have a look at [Akrolyb Interoptability](#Interoptability-with-Akrolyb)***

In [1]:
%matplotlib notebook

%load_ext autoreload

from IPython.core.display import display, HTML
display(HTML("<style>div.output_area pre {white-space: pre;}</style>"))

In [2]:
%autoreload
from collections.abc import Iterable

from core.start import process_files
from utils.formats import *

from core.accumulator import Accumulator
from core.decs import *

from strategies import\
    strings as strings_strategy,\
    methods as methods_strategy,\
    structures as structures_strategy,\
    fields as fields_strategy

from core.strategy_handler import StrategyHandler, FLAG_CLASS, FLAG_METHOD, FLAG_FIELD

from utils import generate
from utils import io_akrolyb

# Loading Androguard

The following code loads the files and starts Androguard

It should support multiprocessing, however the Pipe communication seems to break when transmitting the processed Androguard Objects. I suspect the Object is simply too big for Pickle to serialize or another component in the transmitting chain.

In [3]:
AG_SESSION_FILE = "./Androguard.ag"
MULTIPROCESS_FILES = False  # Currently not working due to serialization issues


# APK Files to load
#file_paths = (
#     "../../../Downloads/com.snapchat.android_10.87.5.69-2074_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_10.88.0.63-2076_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_10.88.5.72-2077_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_10.89.0.66-2078_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_10.89.7.78-2085_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#     "../../../Downloads/com.snapchat.android_11.0.0.72-2086_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#    "../../../Downloads/com.snapchat.android_11.0.1.72-2089_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk",
#    "../../../Downloads/com.snapchat.android_11.0.6.82-2095_minAPI19(arm64-v8a)(nodpi)_apkmirror.com.apk"
#)


file_paths = (
# #     "../../../Downloads/com.instagram.android_143.0.0.25.121-216817305_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
    
# #     "../../../Downloads/com.instagram.android_151.0.0.23.120-232868015_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
# #     "../../../Downloads/com.instagram.android_151.0.0.23.120-232867993_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
# #     "../../../Downloads/com.instagram.android_152.0.0.25.117-234847234_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
# #     "../../../Downloads/com.instagram.android_152.0.0.25.117-234847224_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
# #     "../../../Downloads/com.instagram.android_153.0.0.34.96-236572355_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
# #     "../../../Downloads/com.instagram.android_153.0.0.34.96-236572319_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
# #     "../../../Downloads/com.instagram.android_156.0.0.26.109-240726452_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
# #     "../../../Downloads/com.instagram.android_156.0.0.26.109-240726384_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk",
    
     "../../../Downloads/com.instagram.android_157.0.0.37.120-242168935_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk",
# #     "../../../Downloads/com.instagram.android_157.0.0.37.120-242168922_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk"
    
     "../../../Downloads/com.instagram.android_159.0.0.29.122-244390467_minAPI23(arm64-v8a)(560,640dpi)_apkmirror.com.apk",
#     "../../../Downloads/com.instagram.android_159.0.0.29.122-244390338_minAPI19(armeabi-v7a)(nodpi)_apkmirror.com.apk"
)

In [4]:
(a, d, dx), (a2, d2, dx2) = process_files(*file_paths, MULTIPROCESS_FILES)

Loading Session from Apk at ../../../Downloads/com.instagram.android_157.0.0.37.120-242168935_minAPI23(arm64-v8a)(nodpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


Loading Session from Apk at ../../../Downloads/com.instagram.android_159.0.0.29.122-244390467_minAPI23(arm64-v8a)(560,640dpi)_apkmirror.com.apk


Requested API level 29 is larger than maximum we have, returning API level 28 instead.


# List of Classes, Methods and Variables

Defining Elements that GraphGuard should try to match (obviously requires full class names).
* `named_c_decs` in form of a tuple of Strings
* `named_m_decs` in form of a tuple of MethodDeclarations
* `named_f_decs` in form of a tuple of FieldDeclarations

In [5]:
# Classes of the Methods and Fields are automatically merged with c_decs

named_c_decs = {
    'MAIN_ACTIVITY': 'com.instagram.mainactivity.MainActivity',
    'CLIPITEM': 'X.5EH',
    'COMMENT_LONGCLICK_HANDLER': 'X.3ZZ',
    'DEVELOPEROPTIONSLAUNCHER': 'com.instagram.debug.devoptions.api.DeveloperOptionsLauncher',
    'DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER': 'com.instagram.direct.visual.DirectVisualMessageViewerController',
    'DM_GESTUREDETECTOR': 'X.3OZ',
    'DM_HOLDER': 'X.9pc',
    'DM_REPLY': 'X.3AA',
    'DM_VOICEMESSAGE': 'X.AYP',
    'DMVIDEOPLAYER': 'X.9Ja',
    'FEEDITEM': 'X.1NJ',
    'FRAGMENT': 'X.1IC',
    'HEADERGENERATOR': 'X.0Q4',
    'HTTPREQUESTTASKFACTORY': 'X.0q5',
    'INAPP_NOTIFICATION': 'X.6lv',
    'IGTV_VIEWER_FRAGMENT': 'X.4AJ',
    'REALTIME_CLIENT_MANAGER': 'com.instagram.realtimeclient.RealtimeClientManager',
    'JACKSON_OBJ': 'X.10T',
    'JACKSON_CREATOR': 'X.0i4',
    'LIVE_POST': 'X.26T',
    'MAINFEED': 'X.1Ip',
    'MEDIAITEM': 'X.1NK',
    'MEDIAJSON_HELPER': 'com.instagram.feed.media.Media__JsonHelper',
    'NOTIFICATION': 'X.1jR',
    'NOTIFICATION_HANDLER': 'X.2KH',
    'PHOTOTIMERCONTROLLER': 'X.3Qa',
    'POSTDETAIL': 'X.1WN',
    'POSTGESTURE_SIMPLEPOST': 'X.1sH',
    'POSTGESTURE_CAROUSELIMAGE': 'X.274',
    'POSTGESTURE_CAROUSELVIDEO': 'X.2Gd',
    'POSTVIEW': 'X.1WS',
    'PROFILE': 'X.0kV',
    'PROFILEDETAIL': 'X.3C8',
    'PROFILEPIC_LONGCLICK': 'X.3CF',
    'PROXYGEN_JNI_HANDLER': 'com.facebook.proxygen.JniHandler',
    'REELVIDEOPLAYER': 'X.2CI',
    'REEL_VIEWER_FRAGMENT': 'com.instagram.reels.fragment.ReelViewerFragment',
    'RESPONSE_HANDLER': 'X.0mB',
    'NOTIFICATION_RUNNABLE': 'X.2c8',
    'SERIALSCHEDULER': 'X.0hv',
    'SESSION': 'X.0LH',
    'SIMPLE_EXECUTOR': 'X.4LK',
    'SWIPE_NAVIGATION_CONTAINER': 'com.instagram.ui.swipenavigation.SwipeNavigationContainer',
    'USER': 'X.1Nx',
    'VIDEOSETTINGSMANAGER': 'X.1gG',
    'VOLUME_INDICATOR': 'com.instagram.ui.widget.volume.VolumeIndicator'
}

In [6]:
named_m_decs = {
    'ACTIVITY_ON_DESTROY': MethodDec('com.instagram.mainactivity.MainActivity', 'onDestroy'),
    'CLIPITEM_CONSTRUCTOR': MethodDec('X.5EH', '<init>', 'X.22x', 'X.9Hx', 'X.0LH', 'android.view.View', 'X.5Dw', 'X.5EQ'),
    'COMMENT_ONLONGCLICK': MethodDec('X.3ZZ', 'onLongPress', 'android.view.MotionEvent'),
    'DEVELOPEROPTIONSLAUNCHER_LOADANDLAUNCHDEVELOPEROPTIONS': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.1F7', 'androidx.fragment.app.FragmentActivity', 'X.0LH', 'android.os.Bundle'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHPROJECTCORESWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchProjectEncoreSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0LH'),
    'DEVELOPEROPTIONSLAUNCHER_LAUNCHSTORIESEXPERIMENTSWITCHERTOOL': MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'launchStoriesExperimentSwitcherTool', 'android.content.Context', 'androidx.fragment.app.FragmentActivity', 'X.0LH'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_PAUSE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_RESUME': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'java.lang.String'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_SETSTATE': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0B', skip_params=True),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_STARTVIEW': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A04', 'com.instagram.direct.visual.DirectVisualMessageViewerController'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_ADVANCETONEXT': MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0G', 'com.instagram.direct.visual.DirectVisualMessageViewerController', 'boolean'),
    'DMGESTUREDETECTOR_ONLONGPRESS': MethodDec('X.3OZ', 'onLongPress', 'android.view.MotionEvent'),
    'DMHOLDER_GETCURRENTDMMEDIA': MethodDec('X.9pc', 'A00'),
    'DMREPLYCALLBACK_RUN': MethodDec('X.3AA', 'A0M', 'X.3AA', 'int'),
    'DMREPLY_SETSEEN': MethodDec('X.3AA', 'A0I', 'X.3AA'),
    'DMVIDEOPLAYER_SEEKTO': MethodDec('X.9Ja', 'A02', 'int', 'boolean'),
    'FEEDITEM_JSON_GENERATE': MethodDec('X.1NJ', 'A00', 'X.0iM'),
    'FRAGMENT_GETACTIVITY': MethodDec('X.1IC', 'getActivity'),
    'HEADERGENERATOR_GENERATEFROMDEVICEINFO': MethodDec('X.0Q4', 'A00', 'android.content.Context'),
    'HTTPREQUESTTASKFACTORY_CREATEREQUEST': MethodDec('X.0q5', 'A03', 'java.lang.String', 'X.0LH'),
    'INAPPNOTIFICATION_RUN': MethodDec('X.6lv', 'run'),
    'IGTVVIEWERFRAGMENT_GETMEDIA': MethodDec('X.4AJ', 'A00', 'X.4AJ'),
    'IGTVVIEWERFRAGMENT_UPDATESTATE': MethodDec('X.4AJ', 'A0B', 'X.4AJ'),
    'INDICATEACTIVITY_DO': MethodDec('com.instagram.realtimeclient.RealtimeClientManager', 'sendCommand', 'java.lang.String', 'java.lang.String', 'com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback'),
    'JACKSON_CLOSE': MethodDec('X.10T', 'close'),
    'JACKSON_CREATEGENERATOR': MethodDec('X.0i4', 'A04', 'java.io.OutputStream', 'java.lang.Integer'),
    'JACKSON_JSONFACTORY': MethodDec('X.0i4', '<init>', 'X.DcO'),
    'LIVE_POST_JSON_GENERATE': MethodDec('X.26T', 'A00', 'X.0jB', 'X.26U'),
    'MAINACTIVITY_GETCHILDFRAGMENTMANAGER': MethodDec('com.instagram.mainactivity.MainActivity', 'AL2'),
    'MAINFEED_CONFIGUREACTIONBAR': MethodDec('X.1Ip', 'configureActionBar', 'X.1Hw'),
    'MEDIAITEM_JSON_GENERATE': MethodDec('X.1NK', 'A01', 'X.0iM', 'boolean'),
    'MEDIA_JSON_GENERATE': MethodDec('com.instagram.feed.media.Media__JsonHelper', 'A00', 'X.0jB', 'X.1NK'),
    'NOTIFICATIONINST_GETJSON': MethodDec('X.1jR', 'A01'),
    'NOTIFICATION_SHOW': MethodDec('X.2KH', 'A01', 'android.content.Context', 'java.lang.CharSequence', 'int'),
    'PHOTOTIMERCONTROLLER_INITPICTURECOUNTDOWN': MethodDec('X.3Qa', 'A01'),
    'POSTDETAIL_SETDATA': MethodDec('X.1WN', 'A05', skip_params=True),
    'POSTGESTURE_SIMPLEPOST_ONDOUBLETAP': MethodDec('X.1sH', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELIMAGE_ONDOUBLETAP': MethodDec('X.274', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTGESTURE_CAROUSELVIDEO_ONDOUBLETAP': MethodDec('X.2Gd', 'onDoubleTap', 'android.view.MotionEvent'),
    'POSTVIEW_ADDMEDIA': MethodDec('X.1WS', 'A01', skip_params=True),
    'PROFILEDETAIL_SETDATA': MethodDec('X.3C8', 'A02', skip_params=True),
    'PROFILE_ISSELF': MethodDec('X.0kV', 'A04', 'X.0LH', 'X.0iw'),
    'PROFILE_ONLONGCLICK': MethodDec('X.3CF', 'A00', skip_params=True),
    'PROXYGEN_SENDREQUESTWITHBODY': MethodDec('com.facebook.proxygen.JniHandler', 'sendRequestWithBodyAndEom', 'org.apache.http.client.methods.HttpUriRequest', 'byte[]', 'int', 'int'),
    'REELVIDEOPLAYER_SEEKTO': MethodDec('X.2CI', 'A0T', 'int'),
    'REELVIEWERFRAGMENT_ADVANCETONEXTREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A1I', 'X.1rX'),
    'REELVIEWERFRAGMENT_PAUSEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0n', 'com.instagram.reels.fragment.ReelViewerFragment', 'java.lang.String'),
    'REELVIEWERFRAGMENT_RESUMEREEL': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0R', 'com.instagram.reels.fragment.ReelViewerFragment'),
    'REELVIEWERFRAGMENT_UPDATEMEDIA': MethodDec('com.instagram.reels.fragment.ReelViewerFragment', 'A0y', 'X.1lh', 'X.1rX'),
    'RESPONSE_HANDLER_ONRESPONSE': MethodDec('X.0mB', 'A00', 'X.0mu', 'X.1cq'),
    'REGULARNOTIFICATION_RUN': MethodDec('X.2c8', 'run'),
    'SERIALSCHEDULER_EXECUTE': MethodDec('X.0hv', 'A02', 'X.0tn'),
    'SESSION_GETTOKEN': MethodDec('X.0LH', 'getToken'),
    'SIMPLEEXECUTOR_ONFAIL': MethodDec('X.4LK', 'onFail', 'X.2AT'),
    'SIMPLEEXECUTOR_ONSUCCESS': MethodDec('X.4LK', 'onSuccess', 'java.lang.Object'),
    'SIMPLEXECUTOR_CONSTRUCTOR': MethodDec('X.4LK', '<init>', 'X.0LH'),
    'SWIPENAVIGATIONCONTAINER_ONTOUCHEVENT': MethodDec('com.instagram.ui.swipenavigation.SwipeNavigationContainer', 'onTouchEvent', 'android.view.MotionEvent'),
    'USER_JSON_GENERATE': MethodDec('X.1Nx', 'A03', 'X.0jB', 'X.0iv'),
    'VIDEOSETTINGSMANAGER_ISAUTOPLAYDISABLED': MethodDec('X.1gG', 'A00', 'X.0LH'),
    'VOLUMEINDICATOR_SETPROGRESS': MethodDec('com.instagram.ui.widget.volume.VolumeIndicator', 'A00', 'int', 'int')
}

In [7]:
named_f_decs = {
    'ACTIONBAR_BASEVIEW': FieldDec('X.1Hv', 'A08'),
    'CLIPSHOLDER_MEDIA': FieldDec('X.22x', 'A00'),
    'COMMENTTOUCHHANDLER_COMMENT': FieldDec('X.3ZZ', 'A04'),
    'COMMENT_TEXT': FieldDec('X.1OU', 'A0X'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0D'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_ROOTVIEW': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mRootView'),
    'DIRECTVISUALMESSAGEVIEWCONTROLLER_VIDEOPLAYER': FieldDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'mVideoPlayer'),
    'DMGESTUREDETECTOR_MESSAGEOBJECT': FieldDec('X.3OZ', 'A01'),
    'DMHOLDER_MEDIA': FieldDec('X.9pM', 'A07'),
    'DMVIDEOPLAYER_CONTROLLER': FieldDec('X.9pX', 'A04'),
    'DMVOICEMESSAGE_RAWMEDIA': FieldDec('X.AYP', 'A04'),
    'FEEDITEM_TYPE': FieldDec('X.1NJ', 'A0I'),
    'HTTPREQUESTTASK_EXECUTOR': FieldDec('X.0tm', 'A00'),
    'IGTVVIEWERFRAGMENT_ROOTVIEW': FieldDec('X.4AJ', 'A0E'),
    'MAINFEED_SESSION': FieldDec('X.1Ip', 'A0S'),
    'MEDIANOTIFICATION_LIST': FieldDec('X.1NA', 'A06'),
    'MEDIA_WRAPPER_LIVEMEDIAVAR': FieldDec('X.1rX', 'A09'),
    'MEDIA_WRAPPER_REELMEDIAVAR': FieldDec('X.1rX', 'A08'),
    'NATIVEINPUTSTREAM_READBUFFER': FieldDec('X.1eB', 'A02'),
    'NATIVEINPUTSTREAMHOLDER_NATIVEINPUTSTREAM': FieldDec('X.1qG', 'A01'),
    'NOTIFICATION_INSTANCE': (FieldDec('X.6lv', 'A00'), FieldDec('X.2c8', 'A00')),
    'NOTIFICATION_SESSION': (FieldDec('X.6lv', 'A01'), FieldDec('X.2c8', 'A01')),
    'POSTDETAILUI_PROFILENAME': FieldDec('X.1q3', 'A06'),
    'POST_VIEW_COMMENTBUTTON': FieldDec('X.1q6', 'A02'),
    'PROFILEDETAILUI_BIOTEXT': FieldDec('X.3CB', 'A0N'),
    'PROFILEDETAILUI_FOLLOWER': FieldDec('X.3CB', 'A00'),
    'PROFILEDETAILUI_FOLLOWING': FieldDec('X.3CB', 'A01'),
    'PROFILEDETAILUI_FULLNAME': FieldDec('X.3CB', 'A0A'),
    'PROFILE_PROFILEPIC': FieldDec('X.3CD', 'A09'),
    'PROXYGEN_PROXYGENRESPONSEHANDLER': FieldDec('com.facebook.proxygen.JniHandler', 'mResponseHandler'),
    'PROXYGENRESPONSEHANDLER_READBUFFER': FieldDec('X.1eA', 'A09'),
    'REELVIDEOPLAYER_CONTROLLER': FieldDec('X.3I4', 'A0D'),
    'REEL_VIEWER_FRAGMENT_SESSION': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'A1K'),
    'REEL_VIEWER_FRAGMENT_VIDEOPLAYER': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mVideoPlayer'),
    'REEL_VIEWER_FRAGMENT_VIEWROOT': FieldDec('com.instagram.reels.fragment.ReelViewerFragment', 'mViewRoot'),
    'RESPONSEHANDLER_NATIVEINPUTSTREAMHOLDER': FieldDec('X.1ji', 'A00'),
    'STORYUSER_SESSION': FieldDec('X.1rX', 'A0D')
}

In [8]:
m_decs = tuple(named_m_decs.values())
f_decs = []
for f_dec in named_f_decs.values():
    if isinstance(f_dec, Iterable):
        f_decs.extend(f_dec)
    else:
        f_decs.append(f_dec)
f_decs = tuple(f_decs)

# Add Classes of Methods and Fields to c_decs
c_decs = tuple(map(lambda m: m.class_name, m_decs)) \
        + tuple(map(lambda v: v.class_name, f_decs)) + tuple(named_c_decs.values())

# Processing and Matching

Loading the accumulator, an object that manages all the possible candidates that are matched by the different Matchers, and extracts the matching candidates. It also performs Inner joins on previous candidates to find the exact (or optimal) match.

In [9]:
accumulator = Accumulator()

Resolving the previously defined MethodDecs. If this fails, the MethodDecs are wrong and contain an error. Make sure the method specified with the MethodDec exists.

In [10]:
dec_cas = resolve_classes(dx, c_decs)

r_cas = tuple(dec_cas.values())
r_mas = resolve_methods(m_decs, dec_cas)
r_fas = resolve_fields(f_decs, dec_cas)

print("Resolved all Classes, Methods and Fields")
if False:
    print("", *map(pretty_format_ma, r_mas), sep="\n* ")

# Arguments to provide to Strategies
s_args = (dx, dx2, r_cas, r_mas, r_fas, accumulator)

Resolved all Classes, Methods and Fields


## Strategies & StrategyHandler

Strategies are essentially functions returning candidates for Classes, Methods or Fields. A `strategy` gets registered in the `StrategyHandler` with Flags indicating whether it tries to match Classes, Methods or Fields. If a search is invoked for a matching flag, the strategy gets invoked on the given arguments.

Global Parameters and Rules of the Strategies allow for easily changing criteria, e. g. use a more lenient search and not require strict matches.

In [11]:
strategies = StrategyHandler()


strings_strategy.MAX_USAGE_COUNT_STR = 20
strings_strategy.UNIQUE_STRINGS_MAJORITY = 2 / 3

methods_strategy.MIN_MATCH_POINTS = 2

## String Strategy

### Exact Counter Match

Extracts Strings used either in the given methods directly or in the classes the methods define for both, the old version and the new version. It then compares the Counters for classes and methods and tries to find exact matches between the Counter Objects.

In [12]:
def string_counter_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs, candidates_ms = string_s.compare_counters()
    
    accumulator.add_candidates(candidates_cs, candidates_ms)

strategies.add_strategy(string_counter_strategy, FLAG_CLASS | FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Method. Considering it a match 
	X.1Ip#configureActionBar(X.1Hw) -> X.02c#configureActionBar(X.40J)
+ Found single candidate for Method. Considering it a match 
	X.1jR#A01() -> X.8yq#A01()
+ Found single candidate for Method. Considering it a match 
	X.0Q4#A00(android.content.Context) -> X.0S0#A00(android.content.Context)
+ Found single candidate for Method. Considering it a match 
	com.instagram.realtimeclient.RealtimeClientManager#sendCommand(java.lang.String, java.lang.String, com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback) -> com.instagram.realtimeclient.RealtimeClientManager#sendCommand(java.lang.String, java.lang.String, com.instagram.realtimeclient.RealtimeClientManager$MessageDeliveryCallback)
+ Found single candidate for Method. Considering it a match 
	X.3AA#A0I(X.3AA) -> X.8Ib#A0H(X.8Ib)
+ Found single candidate for Method. Considering it a match 
	X.4LK#onFail(X.2AT) -> X.0lZ#onFail(X.2in)
+ Found single candidate for

### Unique Strings

Gather all Strings that are used only in a single class ("Unique Strings") that we still need to match. Then try to find the matching class by only searching for the Unique Strings.

In [13]:
def unique_strings_strategy(r_cas, r_mas, r_fas):
    string_s = strings_strategy.StringStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = string_s.compare_unique_strings()
    
    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(unique_strings_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

~ Unique String not used in single class anymore. Change! (end_scene)
~ Unique String not used in single class anymore. Change! (media/%s/comment_info/)
.. Considering ambiguous match by UniqueString as match with a majority of 0.67 (LX/0LH; -> LX/0Ly;)
.. Considering ambiguous match by UniqueString as match with a majority of 0.96 (Lcom/instagram/reels/fragment/ReelViewerFragment; -> Lcom/instagram/reels/fragment/ReelViewerFragment;)
+ Found single candidate for Class. Considering it a match! 
	LX/0LH; -> LX/0Ly;
+ Found single candidate for Class. Considering it a match! 
	LX/0hv; -> LX/7mK;
+ Found single candidate for Class. Considering it a match! 
	LX/0tm; -> LX/7eu;
+ Found single candidate for Class. Considering it a match! 
	LX/10T; -> LX/CxS;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/mainactivity/MainActivity; -> Lcom/instagram/mainactivity/MainActivity;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/ui/swipenav

## Structure Strategy

Iterating through every single class and checks for each unmatched class if both have a similar "Profile":
* Number of Methods and Fields
* Types of Fields and Descriptors of Methods

In [14]:
def structure_strategy(r_cas, r_mas, r_fas):
    structure_s = structures_strategy.StructureStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)
    candidates_cs = structure_s.get_exact_structure_matches()

    accumulator.add_candidates(candidates_cs)

strategies.add_strategy(structure_strategy, FLAG_CLASS)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

+ Found single candidate for Class. Considering it a match! 
	LX/3ZZ; -> LX/3To;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher; -> Lcom/instagram/debug/devoptions/api/DeveloperOptionsLauncher;
+ Found single candidate for Class. Considering it a match! 
	LX/3OZ; -> LX/8U5;
+ Found single candidate for Class. Considering it a match! 
	LX/9pc; -> LX/1qJ;
+ Found single candidate for Class. Considering it a match! 
	LX/9Ja; -> LX/2y2;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/feed/media/Media__JsonHelper; -> Lcom/instagram/feed/media/Media__JsonHelper;
+ Found single candidate for Class. Considering it a match! 
	LX/3Qa; -> LX/1DJ;
+ Found single candidate for Class. Considering it a match! 
	LX/1WS; -> LX/3A9;
+ Found single candidate for Class. Considering it a match! 
	LX/0kV; -> LX/0w1;
+ Found single candidate for Class. Considering it a match! 
	LX/3CF; -> LX/3oY;
+ Found 

## Method Strategy

Uses different weighted criteria to get exact or optimal matches. The criteria are:
* Modifiers
* Return Type and Parameter Types
* Length of Bytecode
* References to and from

In [15]:
def method_strategy(r_cas, r_mas, r_fas):
    method_s = methods_strategy.MethodStrategy(dx, dx2, r_cas, r_mas, r_fas, accumulator)

    print("Exact Matching")
    print()

    # Exact Matches
    candidates_ms = method_s.try_resolve_ms(True)
    accumulator.add_candidates(candidates_ms=candidates_ms)

    print()
    print("Non-Exact Matching")
    print()

    method_s.update_matched()
    # Non-Exact Matches by using weights on the criteria
    candidates_ms = method_s.try_resolve_ms(False)
    accumulator.add_candidates(candidates_ms=candidates_ms)
    
strategies.add_strategy(method_strategy, FLAG_METHOD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Exact Matching

Class not resolved for Method X.5EH#<init>(X.22x, X.9Hx, X.0LH, android.view.View, X.5Dw, X.5EQ)
Class not resolved for Method X.0q5#A03(java.lang.String, X.0LH)
Class not resolved for Method X.0i4#A04(java.io.OutputStream, java.lang.Integer)
Class not resolved for Method X.0i4#<init>(X.DcO)
Class not resolved for Method X.1sH#onDoubleTap(android.view.MotionEvent)
Class not resolved for Method X.274#onDoubleTap(android.view.MotionEvent)
Class not resolved for Method X.2Gd#onDoubleTap(android.view.MotionEvent)
+ Found single candidate for Method. Considering it a match 
	X.3ZZ#onLongPress(android.view.MotionEvent) -> X.3To#onLongPress(android.view.MotionEvent)
* Found multiple candidates for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0LH)
* Found multiple candidates for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesE

+ Found single candidate for Method. Considering it a match 
	com.instagram.reels.fragment.ReelViewerFragment#A0y(X.1lh, X.1rX) -> com.instagram.reels.fragment.ReelViewerFragment#A0s(X.0pp, X.18l)
+ Found single candidate for Method. Considering it a match 
	X.0hv#A02(X.0tn) -> X.7mK#A02(X.7mG)
+ Found single candidate for Method. Considering it a match 
	X.0LH#getToken() -> X.0Ly#getToken()
+ Found single candidate for Method. Considering it a match 
	X.1Nx#A03(X.0jB, X.0iv) -> X.FEi#A03(X.CxI, X.FEj)
Could resolve 0 new Classes, 18 new Methods, 0 new Fields.


## Field Strategy

Gathers all fields of matching Class and filters using the following criteria:
* By Type if a matching Type has been found
* By weighted sub-criteria such as:
  * Access Flags
  * Size
  * Number of reads & Number of writes
* By Index if the list of Fields has not changed dramatically

In [16]:
def field_strategy(r_cas, r_mas, r_fas):
    field_s = fields_strategy.FieldStrategy(*s_args)
    
    print("Resolving Types of Fields")
    strategies.invoke_strategies(tuple(field_s.get_types_to_match()))
    print()
    
    candidates_fs = field_s.try_resolve_fs()
    
    accumulator.add_candidates(candidates_fs=candidates_fs)
    
strategies.add_strategy(field_strategy, FLAG_FIELD)
strategies.invoke_strategies(r_cas, r_mas, r_fas, only_new=True)

Resolving Types of Fields
Could resolve 0 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0iv; -> LX/FEj;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/common/ui/widget/imageview/IgImageView; -> Lcom/instagram/common/ui/widget/imageview/IgImageView;
+ Found single candidate for Class. Considering it a match! 
	LX/1P2; -> LX/3AD;
+ Found single candidate for Class. Considering it a match! 
	LX/26U; -> LX/19Q;
Could resolve 4 new Classes, 0 new Methods, 0 new Fields.
+ Found single candidate for Class. Considering it a match! 
	LX/0tt; -> LX/1PV;
+ Found single candidate for Class. Considering it a match! 
	Lcom/instagram/igtv/ui/GestureManagerFrameLayout; -> Lcom/instagram/igtv/ui/GestureManagerFrameLayout;
+ Found single candidate for Class. Considering it a match! 
	Lcom/facebook/proxygen/ReadBuffer; -> Lcom/facebook/proxygen/ReadBuffer;
+ Found single candidate for Class. Considering it a match!

# Results
Display summary and matching pairs.

Output to compatible MethodDec input to update from auto-updated values.

In [17]:
print(len(accumulator.matching_cs), "/", len(named_c_decs), "Classes were matched")
print(len(accumulator.matching_ms), "/", len(m_decs), "Methods were matched")
print(len(accumulator.matching_fs), "/", len(f_decs), "Fields were matched")

print()
print("Matching Classes:")
for c1, c2 in accumulator.matching_cs.items():
    print("•", pretty_format_class(c1), "->", pretty_format_class(c2))

print()
print("Matching Methods:")
for m, ma in accumulator.matching_ms.items():
    print("•", pretty_format_ma(m), "->", pretty_format_ma(ma))

print()
print("Matching Fields:")
for fa, fa2 in accumulator.matching_fs.items():
    print("•", pretty_format_fa(fa), "->", pretty_format_fa(fa2))

68 / 46 Classes were matched
49 / 60 Methods were matched
33 / 39 Fields were matched

Matching Classes:
• X.1Ip -> X.02c
• X.1jR -> X.8yq
• X.0Q4 -> X.0S0
• com.instagram.realtimeclient.RealtimeClientManager -> com.instagram.realtimeclient.RealtimeClientManager
• X.3AA -> X.8Ib
• X.4LK -> X.0lZ
• com.instagram.direct.visual.DirectVisualMessageViewerController -> com.instagram.direct.visual.DirectVisualMessageViewerController
• X.1gG -> X.2Sr
• X.0mB -> X.CRh
• X.3C8 -> X.3oV
• X.6lv -> X.8yK
• X.2c8 -> X.8yL
• X.2CI -> X.ELB
• X.0LH -> X.0Ly
• X.0hv -> X.7mK
• X.0tm -> X.7eu
• X.10T -> X.CxS
• com.instagram.mainactivity.MainActivity -> com.instagram.mainactivity.MainActivity
• com.instagram.ui.swipenavigation.SwipeNavigationContainer -> com.instagram.ui.swipenavigation.SwipeNavigationContainer
• X.1Hv -> X.4dl
• X.1IC -> X.Bk6
• X.1NJ -> X.3AC
• X.1NK -> X.2mu
• X.1Nx -> X.FEi
• X.1OU -> X.2qS
• X.1WN -> X.33y
• X.1eA -> X.CRg
• X.1eB -> X.CRs
• X.1rX -> X.18l
• X.22x -> X.2Cw
• X.26T

• com.instagram.reels.fragment.ReelViewerFragment#A0R(com.instagram.reels.fragment.ReelViewerFragment) -> com.instagram.reels.fragment.ReelViewerFragment#A0P(com.instagram.reels.fragment.ReelViewerFragment)
• com.instagram.reels.fragment.ReelViewerFragment#A0y(X.1lh, X.1rX) -> com.instagram.reels.fragment.ReelViewerFragment#A0s(X.0pp, X.18l)
• X.0hv#A02(X.0tn) -> X.7mK#A02(X.7mG)
• X.0LH#getToken() -> X.0Ly#getToken()
• X.1Nx#A03(X.0jB, X.0iv) -> X.FEi#A03(X.CxI, X.FEj)

Matching Fields:
• LX/22x;#A00 -> LX/2Cw;#A00
• LX/3ZZ;#A04 -> LX/3To;#A04
• Lcom/instagram/direct/visual/DirectVisualMessageViewerController;#A0D -> Lcom/instagram/direct/visual/DirectVisualMessageViewerController;#A0D
• Lcom/instagram/direct/visual/DirectVisualMessageViewerController;#mRootView -> Lcom/instagram/direct/visual/DirectVisualMessageViewerController;#mRootView
• Lcom/instagram/direct/visual/DirectVisualMessageViewerController;#mVideoPlayer -> Lcom/instagram/direct/visual/DirectVisualMessageViewerControlle

In [18]:
generate.generate_m_decs(m_decs, r_mas, accumulator.matching_ms)
generate.generate_c_decs(named_c_decs.values(), r_cas, accumulator.matching_cs)
generate.generate_f_decs(f_decs, r_fas, accumulator.matching_fs)

m_decs = (
    # No match for Method com.instagram.mainactivity.MainActivity#onDestroy(),
    # No match for Method X.5EH#<init>(X.22x, X.9Hx, X.0LH, android.view.View, X.5Dw, X.5EQ),
    MethodDec('X.3To', 'onLongPress', 'android.view.MotionEvent'),
    MethodDec('com.instagram.debug.devoptions.api.DeveloperOptionsLauncher', 'loadAndLaunchDeveloperOptions', 'android.content.Context', 'X.Bk7', 'androidx.fragment.app.FragmentActivity', 'X.0Ly', 'android.os.Bundle'),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0LH),
    # No match for Method com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0LH),
    MethodDec('com.instagram.direct.visual.DirectVisualMessageViewerController', 'A0C', 'com.instagram.direct.visual.DirectVisualMessageViewerCo

In [19]:
"""
c_file = "/home/jaqxues/CodeProjects/IdeaProjects/SnipTools/packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/ClassDeclarations.kt"
m_file = "/home/jaqxues/CodeProjects/IdeaProjects/SnipTools/packimpl/src/main/java/com/jaqxues/sniptools/packimpl/hookdec/MemberDeclarations.kt"
f_file = "/dev/null"
"""


c_file = "/home/marz/StudioProjects/XposedPlayground/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/ClassDecsV8.kt"
m_file = "/home/marz/StudioProjects/XposedPlayground/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/MemberDecsV8.kt"
f_file = "/home/marz/StudioProjects/XposedPlayground/Instaprefs/packimpl/src/main/java/com/marz/instaprefs/packimpl/decs/armv8/VariableDecsV8.kt"


In [20]:
%autoreload

c_txt = io_akrolyb.replace_cs(c_file, accumulator)
m_txt = io_akrolyb.replace_ms(m_file, accumulator, named_m_decs)
f_txt = io_akrolyb.replace_fs(f_file, accumulator, named_f_decs)

No matching Method found for com.instagram.mainactivity.MainActivity#onDestroy()
No matching Method found for X.5EH#<init>(X.22x, X.9Hx, X.0LH, android.view.View, X.5Dw, X.5EQ)
No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchProjectEncoreSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0LH)
No matching Method found for com.instagram.debug.devoptions.api.DeveloperOptionsLauncher#launchStoriesExperimentSwitcherTool(android.content.Context, androidx.fragment.app.FragmentActivity, X.0LH)
No matching Method found for X.0q5#A03(java.lang.String, X.0LH)
No matching Method found for X.0i4#A04(java.io.OutputStream, java.lang.Integer)
No matching Method found for X.0i4#<init>(X.DcO)
No matching Method found for X.1sH#onDoubleTap(android.view.MotionEvent)
No matching Method found for X.274#onDoubleTap(android.view.MotionEvent)
No matching Method found for X.2Gd#onDoubleTap(android.view.MotionEvent)
No matching Method fo

In [21]:
print(c_txt)
print(m_txt)
print(f_txt)

package com.marz.instaprefs.packimpl.decs.armv8

import com.jaqxues.akrolyb.genhook.decs.ClassDec
import com.marz.instaprefs.packimpl.decs.AClassDecs

object ClassDecsV8: AClassDecs() {
    override val MAIN_ACTIVITY = ClassDec("com.instagram.mainactivity.MainActivity")

    override val CLIPITEM = /* TODO */ ClassDec("X.5EH")

    override val COMMENT_LONGCLICK_HANDLER = ClassDec("X.3To")

    override val DEVELOPEROPTIONSLAUNCHER = ClassDec("com.instagram.debug.devoptions.api.DeveloperOptionsLauncher")

    override val DIRECT_VISUAL_MESSAGE_VIEW_CONTROLLER = ClassDec("com.instagram.direct.visual.DirectVisualMessageViewerController")

    override val DM_GESTUREDETECTOR = ClassDec("X.8U5")

    override val DM_HOLDER = ClassDec("X.1qJ")

    override val DM_REPLY = ClassDec("X.8Ib")

    override val DM_VOICEMESSAGE = ClassDec("X.8ZO")

    override val DMVIDEOPLAYER = ClassDec("X.2y2")

    override val FEEDITEM = ClassDec("X.3AC")

    override val FRAGMENT = ClassDec("X.Bk6")

   

}
package com.marz.instaprefs.packimpl.decs.armv8

import android.view.ViewGroup
import android.widget.FrameLayout
import android.widget.ImageView
import android.widget.LinearLayout
import android.widget.TextView
import com.jaqxues.akrolyb.genhook.decs.FieldClass
import com.jaqxues.akrolyb.genhook.decs.VariableDec
import com.marz.instaprefs.packimpl.decs.AVariableDecs
import java.io.InputStream

object VariableDecsV8: AVariableDecs() {
    @FieldClass(["X.1Hv"])
    override val ACTIONBAR_BASEVIEW = /* TODO */ VariableDec<ViewGroup>("A08")

    @FieldClass(["X.2Cw"])
    override val CLIPSHOLDER_MEDIA = VariableDec<Any>("A00")

    @FieldClass(["X.3To"])
    override val COMMENTTOUCHHANDLER_COMMENT = VariableDec<Any?>("A04")

    @FieldClass(["X.1OU"])
    override val COMMENT_TEXT = /* TODO */ VariableDec<String>("A0X")

    @FieldClass(["com.instagram.direct.visual.DirectVisualMessageViewerController"])
    override val DIRECTVISUALMESSAGEVIEWCONTROLLER_DMHOLDER = VariableDec<Any>("A

# Interoptability with Akrolyb

The following Code snippet allows to "extract" MethodDecs from MemberDeclarations in Akrolyb. This automates the process of providing GraphGuard the hooks it should find. Just paste the output (tuple of MethodDecs) in the corresponding section in this Notebook.

## Extract

It is not in Python, since it would require a Kotlin Parser and evaluating imports, variables etc. Just executing the Constructors in Kotlin, then getting the values is much easier than static analysis. 

The `main` function can (and should) be run statically (locally on the machine and not on your Android) to build the list of `MethodDec`s that GraphGuard is supposed to match in an updated build of the target application. Adjust the MemberDeclarations Class to the Class where you declared the `MemberDeclaration`s.


```kotlin
fun main() {
    GraphGuard.printGeneratedDecs(ClassDecs, MemberDecs, VariableDecs)
}
```